In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import *
from scipy.interpolate import Rbf
from numpy import random

### Consider the Vander Pol Oscillator given by the equations:

$\dot{x_1} = 2x_2$

$\dot{x_2} = -0.8x_1+2x_2-10x_1^2x_2+u$

In [ ]:
# define the dynamics
vanderpol = lambda t,x,u : np.asarray([2*x[1],-0.8*x[0]+2*x[1]-10*x[0]**2*x[1]+u])

In [ ]:
dt = 0.01 # sampling time
nstates = 2 # number of states
nctrl = 1 # number of control inputs

In [ ]:
# RK4
k1 = lambda t,x,u: vanderpol(t,x,u)
k2 = lambda t,x,u: vanderpol(t,x+k1(t,x,u)*dt/2,u)
k3 = lambda t,x,u: vanderpol(t,x+k2(t,x,u)*dt/2,u)
k4 = lambda t,x,u: vanderpol(t,x+k1(t,x,u)*dt,u)
rk4_step = lambda t,x,u: x+(dt/6)*(k1(t,x,u)+2*k2(t,x,u)+2*k3(t,x,u)+k4(t,x,u))

In [ ]:
# define the Radial Basis Function that will aid in lifting the states
def rbf(X,C,rbf_type):
    
    Cbig = C ; Y = np.zeros([C.shape[1],X.shape[1]])
    
    for i in range(Cbin.shape[1]):
        
        C = Cbig[:,i]
        C = np.tile(C,X.shape[1])
        r_squared = np.sum((X-C)**2)
        
        y = Rbf(r_squared, mode = rbf_type)
        
        Y[i,:] = y
    
    return Y

In [ ]:
Nrbf = 100 # number of RBF centers
cent = np.random.uniform(0,1,[nstates,Nrbf])*2-1 # generate random RBF centers
rbf_type = 'thin_plate' # specify the type of RBF

# obtain the lifted states
liftFun = lambda xx,cent: np.vstack([xx,rbf(xx,cent,rbf_type)])
# update the total dimension of the lifted state vector
Nlift = Nrbf+nstates

## Collect Data

In [ ]:
Nsim = 200
Ntraj = 1000

# Random forcing
Ubig = 2*np.random.uniform(0,1,[Nsim,Ntraj]) - 1

# Random initial conditions
Xcurrent = (np.random.uniform(0,1,[nstates,Ntraj])*2 - 1)

X = np.empty([nstates,Nsim*Ntraj]); Y = np.empty([nstates,Nsim*Ntraj])
U = []

# collect trajectories
for i in range(Nsim):
    Xnext = rk4_step(0,Xcurrent,Ubig[i,:])
    X[:,i*Ntraj:(i+1)*Ntraj] = Xcurrent
    Y[:,i*Ntraj:(i+1)*Ntraj] = Xnext
    U.append(Ubig[i,:])
    Xcurrent = Xnext

U = np.asarray(U)

## Lift

In [ ]:
Xlift = liftFun(X,cent)
Ylift = liftFun(Y,cent)